## Step 1. Create csv file with photos

(extract coordinates, year, month, date from photo info and puts this data along with photo ID into csv file

In [ ]:
import os
import pandas as pd
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import re
from tqdm import tqdm

# === Параметры ===
ROOT = os.path.abspath("..")  # если скрипт лежит в notebooks/
IMAGE_FOLDER = os.path.join(ROOT, "images")
CSV_FOLDER = os.path.join(ROOT, "csv")
os.makedirs(CSV_FOLDER, exist_ok=True)
CSV_OUTPUT = os.path.join(CSV_FOLDER, "photos.csv")

def convert_to_degrees(v):
    d, m, s = v
    return float(d) + float(m)/60 + float(s)/3600

def extract_gps(image_path):
    try:
        image = Image.open(image_path)
        exif_data = image._getexif()
        if not exif_data:
            return None, None
        gps_raw = {}
        for tag_id, val in exif_data.items():
            tag = TAGS.get(tag_id)
            if tag == "GPSInfo":
                for key in val:
                    gps_tag = GPSTAGS.get(key)
                    gps_raw[gps_tag] = val[key]
        if 'GPSLatitude' in gps_raw and 'GPSLongitude' in gps_raw:
            lat = convert_to_degrees(gps_raw['GPSLatitude'])
            if gps_raw.get('GPSLatitudeRef', 'N') != 'N':
                lat = -lat
            lon = convert_to_degrees(gps_raw['GPSLongitude'])
            if gps_raw.get('GPSLongitudeRef', 'E') != 'E':
                lon = -lon
            return lat, lon
    except Exception as e:
        print(f"❌ Ошибка при обработке {image_path}: {e}")
    return None, None

def extract_date_from_filename(filename):
    match = re.search(r'IMG_(\d{4})(\d{2})(\d{2})', filename)
    if match:
        return int(match.group(1)), int(match.group(2)), int(match.group(3))
    return None, None, None

# === Сканирование
records = []
images = [f for f in os.listdir(IMAGE_FOLDER) if f.lower().endswith(".jpg")]

for fname in tqdm(images, desc="📷 Обработка изображений", ncols=80):
    full_path = os.path.join(IMAGE_FOLDER, fname)
    lat, lon = extract_gps(full_path)
    year, month, day = extract_date_from_filename(fname)
    records.append({
        'filename': fname,
        'folder': os.path.basename(IMAGE_FOLDER),
        'latitude': lat,
        'longitude': lon,
        'year': year,
        'month': month,
        'day': day
    })

# === Сохраняем
df = pd.DataFrame(records)
df.to_csv(CSV_OUTPUT, index=False)
print(f"✅ Сохранено: {CSV_OUTPUT}")

## Generating preview thumbnails in circles of a certain color

In [ ]:
import os
import pandas as pd
from PIL import Image, ImageDraw
from tqdm import tqdm

# === Константы проекта ===
ROOT = "/Users/mloktionov/PycharmProjects/PhotoMaps"
CSV_PATH = os.path.join(ROOT, "csv", "photos.csv")
IMAGE_FOLDER = os.path.join(ROOT, "images")
THUMBNAIL_FOLDER = os.path.join(ROOT, "thumbnails")
SIZE = (64, 64)
BORDER_WIDTH = 4
FORMAT = "PNG"

# === Цветовая палитра по годам ===
YEAR_COLORS = {
    "default_before_2000": "#8c564b",
    "default_after_2031": "#9467bd",
}
palette = [
    "#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b",
    "#e377c2", "#7f7f7f", "#bcbd22", "#17becf", "#aec7e8", "#ffbb78",
    "#98df8a", "#ff9896", "#c5b0d5", "#c49c94", "#f7b6d2", "#c7c7c7",
    "#dbdb8d", "#9edae5", "#393b79", "#637939", "#8c6d31", "#843c39",
    "#7b4173", "#3182bd", "#e6550d", "#31a354", "#756bb1", "#636363",
    "#17becf", "#bcbd22"
]
for i, year in enumerate(range(2000, 2032)):
    YEAR_COLORS[year] = palette[i % len(palette)]

# === Создание папки превью
os.makedirs(THUMBNAIL_FOLDER, exist_ok=True)

# === Загрузка CSV
df = pd.read_csv(CSV_PATH)

# === Функция генерации круглой миниатюры
def create_circular_thumbnail(image_path, output_path, border_color):
    try:
        img = Image.open(image_path).convert("RGBA")
        img = img.resize(SIZE, Image.LANCZOS)

        # Маска круга
        mask = Image.new('L', SIZE, 0)
        draw = ImageDraw.Draw(mask)
        draw.ellipse((0, 0, SIZE[0], SIZE[1]), fill=255)

        # Применяем маску
        result = Image.new('RGBA', SIZE)
        result.paste(img, (0, 0), mask)

        # Обводка
        draw = ImageDraw.Draw(result)
        inset = BORDER_WIDTH // 2
        draw.ellipse(
            (inset, inset, SIZE[0] - inset - 1, SIZE[1] - inset - 1),
            outline=border_color, width=BORDER_WIDTH
        )

        # Сохраняем
        result.save(output_path, FORMAT)
    except Exception as e:
        print(f"❌ Ошибка при обработке {image_path}: {e}")

# === Прогон по всем изображениям
for _, row in tqdm(df.iterrows(), total=len(df), desc="🎨 Генерация превью", ncols=80):
    fname = row["filename"]
    year = int(row["year"]) if not pd.isna(row["year"]) else None

    if year is None:
        color = "#999999"
    elif year < 2000:
        color = YEAR_COLORS["default_before_2000"]
    elif year > 2031:
        color = YEAR_COLORS["default_after_2031"]
    else:
        color = YEAR_COLORS.get(year, "#000000")

    in_path = os.path.join(IMAGE_FOLDER, fname)
    out_name = fname.replace(".jpg", ".png").replace(".JPG", ".png")
    out_path = os.path.join(THUMBNAIL_FOLDER, out_name)

    create_circular_thumbnail(in_path, out_path, color)

print(f"✅ {len(df)} миниатюр сгенерировано в папку: {THUMBNAIL_FOLDER}")

In [ ]:
import os
import zipfile
import pandas as pd
from tqdm import tqdm
from xml.sax.saxutils import escape

# === Константы ===
ROOT = os.getcwd()
CSV_PATH = os.path.join(ROOT, "csv", "photos.csv")
THUMBNAIL_FOLDER = os.path.join(ROOT, "thumbnails")
KMZ_FOLDER = os.path.join(ROOT, "kml")
FULL_IMAGE_BASE_URL = "https://mloktionov.github.io/PhotoMaps/images/"
THUMBNAIL_RELATIVE_PATH = "thumbnails"

# === Подготовка
os.makedirs(KMZ_FOLDER, exist_ok=True)
df = pd.read_csv(CSV_PATH)
df = df.dropna(subset=["latitude", "longitude"])

# === Разбивка по годам
years = sorted(df['year'].dropna().unique().astype(int))

for year in years:
    df_year = df[df['year'] == year]

    kml_parts = [f'''<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2">
  <Document>
    <name>PhotoMaps {year}</name>
''']

    for _, row in tqdm(df_year.iterrows(), total=len(df_year), desc=f"📍 {year}", ncols=80):
        fname = row["filename"]
        lat = row["latitude"]
        lon = row["longitude"]
        month = int(row["month"])
        day = int(row["day"])
        date_label = f"{month:02d}-{day:02d}"

        thumb_name = fname.replace(".jpg", ".png").replace(".JPG", ".png")
        image_url = FULL_IMAGE_BASE_URL + fname
        icon_href = os.path.join(THUMBNAIL_RELATIVE_PATH, thumb_name)

        placemark = f"""
        <Placemark>
          <name>{date_label}</name>
          <Style>
            <IconStyle>
              <scale>1.2</scale>
              <Icon>
                <href>{icon_href}</href>
              </Icon>
            </IconStyle>
          </Style>
          <description><![CDATA[
            <div style="font-family: sans-serif; font-size: 13px;">
              <img src="{image_url}" width="600"><br>
              <i>{escape(fname)}</i>
            </div>
          ]]></description>
          <Point>
            <coordinates>{lon},{lat},0</coordinates>
          </Point>
        </Placemark>
        """
        kml_parts.append(placemark)

    kml_parts.append("  </Document>\n</kml>")

    # Сохраняем doc.kml
    doc_kml_path = os.path.join(KMZ_FOLDER, f"doc_{year}.kml")
    with open(doc_kml_path, "w", encoding="utf-8") as f:
        f.write("\n".join(kml_parts))

    # Упаковка в KMZ
    kmz_path = os.path.join(KMZ_FOLDER, f"PhotoMaps_{year}.kmz")
    with zipfile.ZipFile(kmz_path, "w", zipfile.ZIP_DEFLATED) as kmz:
        kmz.write(doc_kml_path, arcname="doc.kml")
        needed_thumbs = df_year["filename"].str.replace(".jpg", ".png").str.replace(".JPG", ".png").tolist()
        for fname in os.listdir(THUMBNAIL_FOLDER):
            if fname in needed_thumbs:
                full_path = os.path.join(THUMBNAIL_FOLDER, fname)
                arcname = os.path.join("thumbnails", fname)
                kmz.write(full_path, arcname=arcname)

    print(f"✅ Сохранено: {kmz_path}")

# Generate preview thumbnails as jpgs

In [ ]:
import os
from PIL import Image
from tqdm import tqdm

# === Параметры ===
SOURCE_FOLDER = "images"
OUTPUT_FOLDER = "thumbnails"
SIZE = (64, 64)

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

images = [f for f in os.listdir(SOURCE_FOLDER) if f.lower().endswith(".jpg")]

for fname in tqdm(images, desc="📷 Создание превью", ncols=80):
    in_path = os.path.join(SOURCE_FOLDER, fname)
    out_path = os.path.join(OUTPUT_FOLDER, fname)
    try:
        img = Image.open(in_path)
        img.thumbnail(SIZE)
        img = img.convert("RGB")  # на всякий случай
        img.save(out_path, "JPEG")
    except Exception as e:
        print(f"❌ {fname}: {e}")

print(f"✅ Сжато {len(images)} изображений в '{OUTPUT_FOLDER}'")

# generate kmz with jpg preview

In [ ]:
import os
import zipfile
import pandas as pd
from tqdm import tqdm
from xml.sax.saxutils import escape

# === Константы ===
ROOT = os.getcwd()
CSV_PATH = os.path.join(ROOT, "csv", "photos.csv")
KMZ_FOLDER = os.path.join(ROOT, "kml")
FULL_IMAGE_BASE_URL = "https://mloktionov.github.io/PhotoMaps/images/"
THUMBNAIL_BASE_URL = "https://mloktionov.github.io/PhotoMaps/thumbnails/"

os.makedirs(KMZ_FOLDER, exist_ok=True)

# Загрузка CSV
df = pd.read_csv(CSV_PATH)
df = df.dropna(subset=["latitude", "longitude"])

# Группировка по годам
years = sorted(df["year"].dropna().unique().astype(int))

for year in years:
    df_year = df[df["year"] == year]

    kml_parts = [f'''<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2">
  <Document>
    <name>PhotoMaps {year}</name>
''']

    for _, row in tqdm(df_year.iterrows(), total=len(df_year), desc=f"📍 {year}", ncols=80):
        fname = row["filename"]
        lat = row["latitude"]
        lon = row["longitude"]
        month = int(row["month"])
        day = int(row["day"])
        date_label = f"{month:02d}-{day:02d}"

        image_url = FULL_IMAGE_BASE_URL + fname
        thumb_url = THUMBNAIL_BASE_URL + fname

        placemark = f"""
        <Placemark>
          <name>{date_label}</name>
          <Style>
            <IconStyle>
              <scale>1.2</scale>
              <Icon>
                <href>{thumb_url}</href>
              </Icon>
            </IconStyle>
          </Style>
          <description><![CDATA[
            <div style="font-family: sans-serif; font-size: 13px;">
              <img src="{image_url}" width="600"><br>
              <i>{escape(fname)}</i>
            </div>
          ]]></description>
          <Point>
            <coordinates>{lon},{lat},0</coordinates>
          </Point>
        </Placemark>
        """
        kml_parts.append(placemark)

    kml_parts.append("  </Document>\n</kml>")

    # Сохраняем doc.kml
    doc_kml_path = os.path.join(KMZ_FOLDER, f"doc_{year}.kml")
    with open(doc_kml_path, "w", encoding="utf-8") as f:
        f.write("\n".join(kml_parts))

    # Упаковываем doc.kml в KMZ
    kmz_path = os.path.join(KMZ_FOLDER, f"PhotoMaps_{year}.kmz")
    with zipfile.ZipFile(kmz_path, "w", zipfile.ZIP_DEFLATED) as kmz:
        kmz.write(doc_kml_path, arcname="doc.kml")

    print(f"✅ Готово: {kmz_path}")

Вот скрипт для Jupyter Notebook, который:

✅ Разбивает photos.csv по годам
✅ Создаёт отдельный .kml для каждого года
✅ Готовит совместимые KML-файлы для импорта в Google My Maps
✅ В описание вставляет ссылку на полное изображение с GitHub

In [ ]:
import os
import pandas as pd
from xml.dom.minidom import Document
from tqdm import tqdm

# === 🔧 КОНСТАНТЫ ===
ROOT = os.getcwd()
CSV_PATH = os.path.join(ROOT, "csv", "photos.csv")
THUMB_URL_BASE = "https://mloktionov.github.io/PhotoMaps/thumbnails"
IMAGE_URL_BASE = "https://mloktionov.github.io/PhotoMaps/images"
OUTPUT_FOLDER = os.path.join(ROOT, "kml", "mymaps_export")

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# === 📄 Загрузка CSV ===
df = pd.read_csv(CSV_PATH)
df = df.dropna(subset=["latitude", "longitude", "year"])
df["year"] = df["year"].astype(int)

# === 🗂️ Группировка по годам ===
for year, group in tqdm(df.groupby("year"), desc="📦 Генерация файлов по годам"):
    doc = Document()
    kml = doc.createElement("kml")
    kml.setAttribute("xmlns", "http://www.opengis.net/kml/2.2")
    doc.appendChild(kml)

    document = doc.createElement("Document")
    kml.appendChild(document)

    for _, row in group.iterrows():
        placemark = doc.createElement("Placemark")

        # 🏷 Название
        name = doc.createElement("name")
        try:
            month_day = row["filename"][4:12]
            name.appendChild(doc.createTextNode(f"{month_day[4:6]}-{month_day[6:]}"))
        except:
            name.appendChild(doc.createTextNode(row["filename"]))
        placemark.appendChild(name)

        # 🔗 Описание (ссылка на полное изображение)
        desc = doc.createElement("description")
        img_url = f"{IMAGE_URL_BASE}/{row['filename']}"
        desc.appendChild(doc.createTextNode(f"📷 {img_url}"))
        placemark.appendChild(desc)

        # 📍 Геопозиция
        point = doc.createElement("Point")
        coords = doc.createElement("coordinates")
        coords.appendChild(doc.createTextNode(f"{row['longitude']},{row['latitude']},0"))
        point.appendChild(coords)
        placemark.appendChild(point)

        document.appendChild(placemark)

    # 💾 Сохраняем файл
    out_path = os.path.join(OUTPUT_FOLDER, f"mymaps_{year}.kml")
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(doc.toprettyxml(indent="  "))

print(f"✅ Готово: KML-файлы для My Maps в {OUTPUT_FOLDER}")

In [ ]:
----

KML - files with PNG previews round with Alpha channel

In [ ]:
----

In [ ]:
import os
from PIL import Image, ImageDraw
from tqdm import tqdm
import pandas as pd

# === 🛠️ Константы ===
ROOT = os.getcwd()
IMAGES_FOLDER = os.path.join(ROOT, "images")
THUMBNAILS_FOLDER = os.path.join(ROOT, "thumbnails")
CSV_PATH = os.path.join(ROOT, "csv", "photos.csv")
SIZE = (64, 64)
BORDER_WIDTH = 4

# 🎨 Цветовая палитра по годам
YEAR_COLORS = {
    "before_2000": "#888888",
    2000: "#e6194B", 2001: "#f58231", 2002: "#ffe119", 2003: "#bfef45",
    2004: "#3cb44b", 2005: "#42d4f4", 2006: "#4363d8", 2007: "#911eb4",
    2008: "#f032e6", 2009: "#a9a9a9", 2010: "#fabed4", 2011: "#ffd8b1",
    2012: "#dcbeff", 2013: "#aaffc3", 2014: "#e6beff", 2015: "#9A6324",
    2016: "#469990", 2017: "#800000", 2018: "#808000", 2019: "#000075",
    2020: "#aaffc3", 2021: "#fffac8", 2022: "#ffd8b1", 2023: "#fabed4",
    2024: "#e6194B", 2025: "#3cb44b", 2026: "#ffe119", 2027: "#4363d8",
    2028: "#f032e6", 2029: "#42d4f4", 2030: "#bfef45"
}

# 📁 Подготовка
os.makedirs(THUMBNAILS_FOLDER, exist_ok=True)
df = pd.read_csv(CSV_PATH)

# 🌀 Генерация
for _, row in tqdm(df.iterrows(), total=len(df), desc="🌀 Генерация PNG", ncols=80):
    fname = row["filename"]
    year = int(row["year"]) if row["year"] >= 2000 else "before_2000"
    in_path = os.path.join(IMAGES_FOLDER, fname)
    out_path = os.path.join(THUMBNAILS_FOLDER, os.path.splitext(fname)[0] + ".png")

    try:
        img = Image.open(in_path).convert("RGBA")
        img.thumbnail(SIZE, Image.LANCZOS)

        # Центрированное изображение на прозрачном фоне
        padded = Image.new("RGBA", SIZE, (0, 0, 0, 0))
        offset = ((SIZE[0] - img.size[0]) // 2, (SIZE[1] - img.size[1]) // 2)
        padded.paste(img, offset)

        # Маска круга
        mask = Image.new("L", SIZE, 0)
        draw_mask = ImageDraw.Draw(mask)
        draw_mask.ellipse((0, 0) + SIZE, fill=255)

        # Цвет рамки по году
        border_color = YEAR_COLORS.get(year, "#999999")

        # Финальный PNG с рамкой
        output = Image.new("RGBA", SIZE, (0, 0, 0, 0))
        draw = ImageDraw.Draw(output)
        draw.ellipse(
            (0, 0, SIZE[0] - 1, SIZE[1] - 1),
            outline=border_color,
            width=BORDER_WIDTH
        )
        output.paste(padded, (0, 0), mask)
        output.save(out_path, "PNG")

    except Exception as e:
        print(f"❌ Ошибка: {fname} — {e}")

print(f"\n✅ Готово! PNG-файлы сохранены в '{THUMBNAILS_FOLDER}'")

In [ ]:
import os
import pandas as pd
from xml.sax.saxutils import escape
from tqdm import tqdm

# === 🔧 Константы ===
ROOT = os.getcwd()
CSV_PATH = os.path.join(ROOT, "csv", "photos.csv")
THUMBNAILS_URL = "https://mloktionov.github.io/PhotoMaps/thumbnails"
KML_OUTPUT_DIR = os.path.join(ROOT, "kml")

# 📁 Убедимся, что папка есть
os.makedirs(KML_OUTPUT_DIR, exist_ok=True)

# 📄 Загрузка данных
df = pd.read_csv(CSV_PATH)
df = df.dropna(subset=["latitude", "longitude"])

# Группировка по годам
groups = df.groupby("year")

for year, group in groups:
    kml_name = f"doc_{year}.kml"
    kml_path = os.path.join(KML_OUTPUT_DIR, kml_name)

    placemarks = []
    for _, row in tqdm(group.iterrows(), total=len(group), desc=f"📍 {year}", ncols=80):
        fname = row["filename"]
        lat = row["latitude"]
        lon = row["longitude"]
        month = str(row["month"]).zfill(2)
        day = str(row["day"]).zfill(2)
        label = f"{month}-{day}"
        thumb_url = f"{THUMBNAILS_URL}/{os.path.splitext(fname)[0]}.png"
        full_url = f"https://mloktionov.github.io/PhotoMaps/images/{fname}"

        placemark = f"""
<Placemark>
  <name>{escape(label)}</name>
  <Style>
    <IconStyle>
      <scale>1.0</scale>
      <Icon>
        <href>{thumb_url}</href>
      </Icon>
    </IconStyle>
  </Style>
  <description><![CDATA[
    <img src="{thumb_url}" width="128"/><br/>
    <a href="{full_url}">{fname}</a>
  ]]></description>
  <Point>
    <coordinates>{lon},{lat},0</coordinates>
  </Point>
</Placemark>
"""
        placemarks.append(placemark.strip())

    # 📦 Обёртка в документ
    kml_content = f"""<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2">
<Document>
  <name>PhotoMap {year}</name>
  {''.join(placemarks)}
</Document>
</kml>
"""

    # 💾 Сохраняем
    with open(kml_path, "w", encoding="utf-8") as f:
        f.write(kml_content)

print(f"\n✅ Сгенерированы KML-файлы по годам в '{KML_OUTPUT_DIR}'")

In [ ]:
import os
print("📍 Текущий путь:", os.getcwd())
print("📂 Содержимое:", os.listdir())

In [ ]:
import os
print(os.listdir("csv"))

In [ ]:
import os
os.path.abspath("csv/photos.csv")

In [4]:
import pandas as pd

df = pd.read_csv("csv/photos.csv")
df_valid = df.dropna(subset=["latitude", "longitude"])
df_sample = df_valid.sample(5, random_state=42)

print(df_sample)

                           filename  folder   latitude  longitude  year  \
229  IMG_20240831_122905034_HDR.jpg  images  50.829571  15.542301  2024   
71   IMG_20220716_124452201_HDR.jpg  images  51.113235  17.026369  2022   
11       IMG_20240518_134431859.jpg  images  51.788551  16.670093  2024   
214         IMG_20220531_125005.jpg  images  50.242031  16.852028  2022   
17   IMG_20221113_133142300_HDR.jpg  images  51.209101  16.164603  2022   

     month  day  
229      8   31  
71       7   16  
11       5   18  
214      5   31  
17      11   13  
